In [1]:
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from sklearn.pipeline import Pipeline

C:\Users\kkyko\AppData\Local\Temp\ipykernel_32256\4012798341.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import numpy as np, pandas as pd


In [2]:
df=pd.read_csv("D:\Work\Personal Project\Amazon Sales Analysis\model data.csv")
df.head()

,Category,Discounted Price,Actual Price,discount_percentage,rating,rating_count
0,Computers & Accessories,399.0,1099.0,0.64,4.2,24269
1,Computers & Accessories,199.0,349.0,0.43,4.0,43994
2,Computers & Accessories,199.0,1899.0,0.90,3.9,7928
3,Computers & Accessories,329.0,699.0,0.53,4.2,94363
4,Computers & Accessories,154.0,399.0,0.61,4.2,16905


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Category             1415 non-null   object 
 1   Discounted Price     1415 non-null   float64
 2   Actual Price         1415 non-null   float64
 3   discount_percentage  1415 non-null   float64
 4   rating               1415 non-null   float64
 5   rating_count         1415 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 66.5+ KB


Data cleaning

In [4]:
# price contains '₹', need to convert to string and remove it, then convert back to float
# ds.discounted_price=ds.discounted_price.str.replace('₹','')
# ds.actual_price=ds.actual_price.str.replace('₹','')

# ds0=ds.copy()  
# ds0.rating.replace('|',float(4.0),inplace=True)
# ds1=ds0.dropna(axis=0)

# # review title and review content are dropped, but they may contain useful data to rating prediction
# drop_feature=['product_name','img_link','product_link','product_id','about_product','user_id','user_name','review_id','review_title','review_content']

# ds1=ds1.drop(drop_feature,axis=1)

y=df.rating
X=df.drop(['rating'],axis=1)

# # remove all the sub-categories

# for c in X.index:
#     X.loc[c,'category']=X.loc[c,'category'].split("|")[0]
    
print(X['Category'].value_counts())

Category
Electronics                491
Home & Kitchen             445
Computers & Accessories    441
Office Products             31
Musical Instruments          2
Home Improvement             2
Toys & Games                 1
Car & Motorbike              1
Health & PersonalCare        1
Name: count, dtype: int64


In [13]:
df.describe()

,Discounted Price,Actual Price,discount_percentage,rating,rating_count
count,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000
mean,3188.973449,5552.875110,0.475993,4.094417,11953.164664
std,7049.236320,11041.365276,0.216845,0.292415,17620.520703
min,39.000000,39.000000,0.000000,2.000000,2.000000
25%,319.500000,800.000000,0.320000,4.000000,1094.500000
50%,799.000000,1690.000000,0.500000,4.100000,4768.000000
75%,2056.000000,4499.500000,0.630000,4.300000,15189.000000
max,77990.000000,139900.000000,0.940000,5.000000,98250.000000


In [5]:
from sklearn.model_selection import train_test_split,cross_val_score,KFold
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=10)

In [6]:
import sklearn; print(sklearn.__version__)

1.4.0


In [8]:
# OneHotEncoding

from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(pd.DataFrame(X_train['Category'],columns=['Category'])))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(pd.DataFrame(X_valid['Category'],columns=['Category'])))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop('Category', axis=1)
num_X_valid = X_valid.drop('Category', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

OH_X_train_1=pd.DataFrame(OH_X_train)
OH_X_valid_1=pd.DataFrame(OH_X_valid)

# OH_X_train_1.iloc[:, 4] = OH_X_train_1.iloc[:, 4].toarray()
# OH_X_valid_1.iloc[:, 4] = OH_X_valid_1.iloc[:, 4].toarray()

OH_X_train_1=OH_X_train_1.astype(float)
OH_X_valid_1=OH_X_valid_1.astype(float)


try random forest to see

In [9]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,accuracy_score,mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=10)
model.fit(OH_X_train_1,y_train)

#Mean Absolute Error
preds = model.predict(OH_X_valid_1)

print('MAE for RF is: {}'.format(mean_absolute_error(y_valid, preds)))

MAE for RF is: 0.1851484098939929


cross validation

In [10]:

# Cross validation

def get_score(n_estimators,X,y):

    my_pipeline = Pipeline(steps=[
                              ('model', RandomForestRegressor(n_estimators=n_estimators,
                                                              random_state=10))
                             ])

    scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
    return scores.mean()

print('cross validation score is:{}'.format(get_score(100,OH_X_train_1,y_train)))

cross validation score is:0.18591256871077166


Random Forest, gradient Boost and Ridge

In [11]:
from sklearn.ensemble import StackingClassifier,StackingRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

models=[
    ('rf',RandomForestRegressor(n_estimators=100,random_state=10)), #n_estimaters = learning steps, randome_Ste = no of decison trees
    ('gb',GradientBoostingRegressor(n_estimators=100,random_state=10)), 
]

meta_es=Ridge(alpha=1.0)
stacking_m=StackingRegressor(estimators=models,final_estimator=meta_es)

#  change the paramaters to see which one is the best

param_grid={
    'rf__n_estimators':[100],
    'rf__max_depth':[None],
    'gb__n_estimators':[100],
    'gb__learning_rate':[0.1],
    'gb__max_depth':[3],
    'final_estimator__alpha':[1.0]

}
grid_search=GridSearchCV(stacking_m,param_grid, cv=5, scoring='neg_mean_squared_error',n_jobs=8)

grid_search.fit(OH_X_train_1,y_train)
predicts= grid_search.predict(OH_X_valid_1)

print('MAE is: {}'.format(mean_squared_error(y_valid, predicts)))

print(grid_search.best_params_)

MAE is: 0.07739174067413199
{'final_estimator__alpha': 1.0, 'gb__learning_rate': 0.1, 'gb__max_depth': 3, 'gb__n_estimators': 100, 'rf__max_depth': None, 'rf__n_estimators': 100}


In [12]:
kf=KFold(n_splits=5,shuffle=True,random_state=10)
cvs=cross_val_score(stacking_m,OH_X_train_1,y_train,cv=kf,scoring='neg_mean_absolute_error')
print('stacking model score:{}'.format(abs(cvs).mean()))

stacking model score:0.191735392022308


In [16]:
print(predicts)

[3.93702495 4.17840515 4.08225944 4.27387211 4.10310254 3.83841744
 4.16705088 4.2127395  4.22415166 4.14670805 3.89157679 3.63583866
 3.98542261 4.05552542 4.05810339 4.13326152 4.25236478 3.9404612
 4.23323823 4.22916767 4.24301833 4.25166067 4.11418267 4.27776152
 4.24495153 4.2108395  4.13963324 4.12144678 4.21847206 4.22776032
 4.10523717 4.03925002 4.18500761 4.10662201 4.01192886 4.22271606
 4.01150722 4.19202697 3.85124486 4.11545802 4.23334885 4.15636361
 4.22611128 4.03761304 4.25413864 4.19662798 3.97479753 4.11361231
 3.91153824 4.1467155  4.30068224 4.14837152 4.04527687 4.38999157
 4.12428333 4.14205752 4.1839809  4.22902894 4.09322424 4.01612274
 4.18101136 4.25763943 4.19127342 4.02532608 4.03725325 4.15712882
 4.08116281 4.11007452 3.93674809 4.02863227 4.02974201 4.16450522
 4.03046705 4.14146872 4.11310871 4.17224041 3.98388444 4.21841135
 3.96791517 4.12428333 4.09085478 4.0992681  3.93150899 4.27521492
 4.16954389 4.03913311 4.03394947 4.20378922 4.10856663 3.93214